# Group 44 Final Project CS109B - Twitter
## Tweet Database Generator



**Spring 2019**<br/>
**Authors**: Daniel Barjum, Yalda Amini, João Araújo

**Description**
This python code generates a single database of all the tweets obtained from various data sources. We generate a single database as it is needed in order to be passed onto our neural network. The tweets database comes from researcher's database that we were granted access on behalf of the researchers to use for this project. Information on the dataset can be obtained here: http://mib.projects.iit.cnr.it/dataset.html

The tweets database contains multiple information for each tweet. Some data is useful, some is not. This code analyzes this data and removes any variables that we deemed were not necesarry for use in predicting whether an acocunt is a bot or not.

<hr style="height:2pt">

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#Define paths to files where data is stored

path = os.getcwd()+'/data/datasets_full/'

folders = ['genuine_accounts/', 'social_spambots_1/', 'social_spambots_2/', 
           'social_spambots_3/', 'traditional_spambots_1/', 'traditional_spambots_2/',
           'traditional_spambots_3/', 'traditional_spambots_4/', 'fake_followers/']

file_names = ['users.csv', 'tweets.csv']

The data containes differing types variable types for some columns which difficults and slows the process of reading into a pandas dataframe. Here we define how some columns should be read into pandas in order to avoid warning and speed up the process a bit. 

Some columns, although numerical in nature, were coerced into strings as they are more useful in that manner. For example, columns that have a user id were changed into a string instead of an int or float as we can use this number if we need to request information from Twitter's API which would prefer to see strings.

In [3]:
#define datatype for some columns to speed up process and avoid warnings when reading raw data
type_dic = {'user_id': str, 'id': str, 'contributors': str,
            'crawled_at': str, 'in_reply_to_screen_name': str,
            'place': str, 'retweeted_status_id': str, 
            'in_reply_to_screen_name': str, 'in_reply_to_status_id': str,
            'in_reply_to_user_id': str, 'updated': str, 'created_at': str}

Read in databases and append each one into a single database. Show number of datapoints being read from each database and final count of observations.

The try-except is there to catch any errors from reading the files or if files do not exits. We discovered that some files were missing from the researchers data, so we just ignore these files during the reading

In [7]:
df = pd.DataFrame()

for f in folders:
    try:
        temp_df = pd.read_csv(path+f+file_names[1], dtype=type_dic)
        print('File {0} has shape of: {1}'.format(f+file_names[1], temp_df.shape))
        df = df.append(temp_df, ignore_index=True, sort=True)
    except FileNotFoundError:
        print('File {:s} not found, ignoring'.format(f+file_names[1])) 

print('final tweet database is of shape {0}'.format(df.shape))

File genuine_accounts/tweets.csv has shape of: (2839362, 25)
File social_spambots_1/tweets.csv has shape of: (1610034, 25)
File social_spambots_2/tweets.csv has shape of: (428542, 25)
File social_spambots_3/tweets.csv has shape of: (1418557, 25)
File traditional_spambots_1/tweets.csv has shape of: (145094, 25)
File traditional_spambots_2/tweets.csv not found, ignoring
File traditional_spambots_3/tweets.csv not found, ignoring
File traditional_spambots_4/tweets.csv not found, ignoring
File fake_followers/tweets.csv has shape of: (196027, 23)
final tweet database is of shape (6637616, 25)


We noticed that some columns contain no information at all. This next section of code checks for columns that contain no information and drops these columns from the dataframe.

In [8]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

empty_cols = []
for c in df.select_dtypes(include=numerics).columns:
    if len(np.isnan(df[c].unique()))==1 & np.isnan(df[c].unique())[0]:
        empty_cols.append(c)

df = df.drop(empty_cols, axis=1)
print('The following columns only contained nulls: {0}, these have been dropped'.format(empty_cols))

The following columns only contained nulls: ['favorited', 'geo', 'retweeted'], these have been dropped


In [9]:
df.shape

(6637616, 22)

In [10]:
df.describe()

,favorite_count,num_hashtags,num_mentions,num_urls,possibly_sensitive,reply_count,retweet_count,truncated
count,6.637615e+06,6.637615e+06,6.637615e+06,6.637615e+06,26812.0,6.637615e+06,6.637615e+06,753.0
mean,2.352860e+00,1.561749e-01,3.908975e-01,2.003855e-01,1.0,2.848357e-02,3.832842e+02,1.0
std,3.313966e+02,5.913658e-01,7.311432e-01,4.062391e-01,0.0,1.474201e+01,1.100351e+04,0.0
min,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,1.0
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,1.0
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,1.0
75%,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,1.0
max,1.353000e+05,2.800000e+01,1.900000e+01,6.000000e+00,1.0,2.751600e+04,3.350111e+06,1.0


we notice from the describe above that some columns have very few observations (possibly_sensitive and truncated). Reading Twitter's API pages about these two variables, we deemed that it was safe to drop this variable for analysis as 1) it has few observations, and 2) the values are not very useful as they proxy other variables.

See https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html for information about variables

In [12]:
df = df.drop(['possibly_sensitive','truncated'], axis=1)

In [13]:
df.head()

,contributors,crawled_at,created_at,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,num_hashtags,num_mentions,num_urls,place,reply_count,retweet_count,retweeted_status_id,source,text,timestamp,updated,user_id
0,NaN,2015-05-01 12:57:19,Fri May 01 00:18:11 +0000 2015,0.0,593932392663912449,NaN,0,0,0.0,1.0,0.0,NaN,0.0,1.0,593932168524533760,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",RT @morningJewshow: Speaking about Jews and co...,2015-05-01 02:18:11,2015-05-01 12:57:19,678033
1,NaN,2015-05-01 12:57:19,Thu Apr 30 21:50:52 +0000 2015,0.0,593895316719423488,NaN,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This age/face recognition thing..no reason pla...,2015-04-30 23:50:52,2015-05-01 12:57:19,678033
2,NaN,2015-05-01 12:57:19,Thu Apr 30 20:52:32 +0000 2015,0.0,593880638069018624,NaN,0,0,2.0,0.0,0.0,NaN,0.0,0.0,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Only upside of the moment I can think of is th...,2015-04-30 22:52:32,2015-05-01 12:57:19,678033
3,NaN,2015-05-01 12:57:19,Thu Apr 30 18:42:40 +0000 2015,1.0,593847955536252928,NaN,0,0,2.0,0.0,0.0,NaN,0.0,2.0,0,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",If you're going to think about+create experien...,2015-04-30 20:42:40,2015-05-01 12:57:19,678033
4,NaN,2015-05-01 12:57:19,Thu Apr 30 18:41:36 +0000 2015,0.0,593847687847350272,NaN,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",Watching a thread on FB about possible future ...,2015-04-30 20:41:36,2015-05-01 12:57:19,678033


We also notice that columns 'contributors', 'in_reply_to_screen_name', and 'place' appear to have lots of NaNs, let's look at the columns to see if they contain useful information

In [25]:
for c in ['contributors', 'in_reply_to_screen_name', 'place']:
    print('column {:s} has {:d} unique values:'.format(c, len(df[c].unique())))
    print(df[c].unique())
    print()

column contributors has 1 unique values:
[nan]

column in_reply_to_screen_name has 271336 unique values:
[nan 'thelancearthur' 'wkamaubell' ... 'QueenBitchEnt' 'QBLilKim'
 'TokyozFinest1']

column place has 3191 unique values:
[nan 'Tucson, AZ' 'Casas Adobes, AZ' ... 'Cártama, Malaga'
 'Chalco, Messico' 'Universiti Multimedia, Bukit Baru']



ok, seems like 'contributors' column is empty so we will remove this column. The other two columns do seem to contain information about tweets and we will keep them for now.

In [26]:
df = df.drop('contributors', axis=1)

Let's check if some columns that should contain a numerical value contain any Null values.

In [40]:
for c in df.columns:
    if df[c].isna().sum() > 0:
        print('column {:s} has {:d} null values'.format(c, df[c].isna().sum()))

column crawled_at has 196028 null values
column created_at has 1 null values
column favorite_count has 1 null values
column in_reply_to_screen_name has 5598482 null values
column in_reply_to_status_id has 1 null values
column in_reply_to_user_id has 1 null values
column num_hashtags has 1 null values
column num_mentions has 1 null values
column num_urls has 1 null values
column place has 6508965 null values
column reply_count has 1 null values
column retweet_count has 1 null values
column retweeted_status_id has 196028 null values
column source has 73 null values
column text has 13007 null values
column timestamp has 1 null values
column updated has 196028 null values
column user_id has 1 null values


We notice a few things here. There are several columns with only 1 null value, and some columns with many null values. Let's explore further to determine if we can fill this missing data or if we can drop them.

We will first check all columns that contain only 1 null, maybe they all point the the exact same observation. if this is the case, then we could simply delete this observation and we would only loose a single data point.

In [50]:
single_nulls = ['created_at', 'favorite_count', 'in_reply_to_status_id',
                'in_reply_to_user_id', 'num_hashtags', 'num_mentions', 
                'num_urls', 'reply_count', 'retweet_count', 'timestamp', 'user_id']

for c in single_nulls:
    print('index {:d} is where the null is for column {:s}'.format(df.loc[df[c].isna()].index[0],c))

index 2839361 is where the null is for column created_at
index 2839361 is where the null is for column favorite_count
index 2839361 is where the null is for column in_reply_to_status_id
index 2839361 is where the null is for column in_reply_to_user_id
index 2839361 is where the null is for column num_hashtags
index 2839361 is where the null is for column num_mentions
index 2839361 is where the null is for column num_urls
index 2839361 is where the null is for column reply_count
index 2839361 is where the null is for column retweet_count
index 2839361 is where the null is for column timestamp
index 2839361 is where the null is for column user_id


as expected, they all correspond to same observation, so lets delete this observation

In [51]:
df = df.drop(df.loc[df.created_at.isna()].index[0], axis=0)

columns 'crawled_at'and 'updated' were columns added by the researchers. They used twitter crawlers to collect information for this database. These columns do not belong to information about users that can be collected through Twitter's API, hence we will drop these two columns.

In [61]:
df = df.drop(['crawled_at', 'updated'], axis=1)

In [62]:
for c in df.columns:
    if df[c].isna().sum() > 0:
        print('column {:s} has {:d} null values'.format(c, df[c].isna().sum()))

column in_reply_to_screen_name has 5598481 null values
column place has 6508964 null values
column retweeted_status_id has 196027 null values
column source has 72 null values
column text has 13006 null values
